## Welcome to the Linear_Regression notebook

Goals of this notebook:
* Use Pandas and Scikit-Learn to train a Linear Regression Model on PrEP users(per NJ county per year), to estimate prep users up until 2030 (11 years into the future)

## Import Necessary Libraries

In [69]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline


## Load in the Dataset

In [70]:
# Primary key = GEOID, year

pp = pd.read_csv('MIS_data.csv')

print(pp.columns)

#remove the empty columns (which exist due to A-Z naming of columns)
pp = pp[['GEO ID', 'County', 'State', 'Year', 'County PrEP Users',
       'HIV New Incidences', 'Census Population Estimates',
       'Median Household Income', 'Unemployment Rate', 'PnR']]

print(pp.columns)

#pp.head()

pp_PrEP = pp[['GEO ID', 'Year', 'County PrEP Users']]

pp_PrEP.groupby(['Year', 'GEO ID', 'County PrEP Users'])['County PrEP Users']
pp_PrEP.head()


#pp_PrEP.plot(kind='line')

Index(['GEO ID', 'County', 'State', 'Year', 'County PrEP Users',
       'HIV New Incidences', 'Census Population Estimates',
       'Median Household Income', 'Unemployment Rate', 'PnR', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22',
       'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25'],
      dtype='object')
Index(['GEO ID', 'County', 'State', 'Year', 'County PrEP Users',
       'HIV New Incidences', 'Census Population Estimates',
       'Median Household Income', 'Unemployment Rate', 'PnR'],
      dtype='object')


,GEO ID,Year,County PrEP Users
0,34001,2012,14
1,34003,2012,34
2,34005,2012,18
3,34007,2012,30
4,34009,2012,5
